#### <strong><font color=cornflowerblue>第四次作品：深度學習影像去模糊模型之訓練流程設計與實作 </font></strong>
學號：711331135

姓名：王宣懿

<hr>

#### <strong>作品目標</strong>：本作品旨在建立並訓練一套高效的影像去模糊模型，透過深度學習技術還原模糊影像的清晰度。首先，從資料集（如 T91、Set5、Set14）中讀取清晰圖像，將其裁切成指定大小的小圖塊（patch），並對每個 patch 套用高斯模糊產生對應的模糊版本，形成成對的訓練資料。這些模糊／清晰影像對依據訓練與驗證需求儲存於不同資料夾，作為模型學習的基礎。接著，透過 PyTorch 架構多種影像去模糊模型，結合自定義 Dataset 類別載入資料、設定損失函數與訓練超參數，完成模型訓練與評估流程。本作品亦設計兩階段訓練策略，先以 Set5 與 Set14 等簡單資料集進行預訓練，協助模型學習基礎特徵擷取能力，再以 T91 資料集進行微調，提升模型在真實模糊影像上的表現。最終透過 PSNR 等指標以及視覺化結果，評估模型效能還原品質與實際去模糊效果。



<hr>

#### <strong>載入套件</strong>：

In [ ]:
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import patchify
import numpy as np
import matplotlib.gridspec as gridspec
import glob as glob
import os
import cv2

<hr>
<hr>

#### **<font color=cornflowerblue>資料預處理：</font>**  
建立去模糊模型所需的訓練與驗證資料集，透過影像裁切與模糊處理產生成對的清晰與模糊影像 patch。從數個資料夾（如 T91、Set5、Set14）讀取清晰影像，依指定大小切割成小圖塊，並對每個 patch 套用高斯模糊產生對應的模糊版本。最終，將這些 patch 分別儲存為清晰版本（作為標籤）與模糊版本（作為輸入），並依據訓練與驗證資料來源存入不同資料夾，供後續模型訓練與測試使用。



#### (1) 資料預處理參數設定（Patch 切割與高斯模糊）

In [ ]:
DIR_PATH = "C:/Users/User/Downloads/DeblurCNN_2025_v1.1"
SHOW_PATCHES = False
STRIDE = 96 # Stride for the patches.
PATCH_SIZE = 96 # Size of the patches.
GAUSSIAN_KERNEL = (5, 5) # Kernel size for Gaussian blur.
SIGMA_X = 0 # Sigma for Gaussian blur. 0 means it is calculated from the kernel size.


#### (2) T91 資料集（ 訓練用 ）影像切割與模糊處理說明
**注意事項：**
- T91_sharp_patches ： 存放清晰原始的影像 patch ， 作為驗證階段的標籤資料。
- T91_blurred_patches : 存放經高斯模糊處理後的 patch ， 作為模型的輸入資料。
- 每個輸出影像檔案皆採用統一的五位數純數字命名（如 00001.png），這種命名方式有助於PyTorch 資料讀取流程 。

In [ ]:

# 設定資料夾
input_dir = DIR_PATH + "inputs/T91"
sharp_out_dir = DIR_PATH + "inputs/T91_sharp_patches"
blur_out_dir = DIR_PATH + "inputs/T91_blurred_patches"


# 建資料夾
os.makedirs(sharp_out_dir, exist_ok=True)
os.makedirs(blur_out_dir, exist_ok=True)

count = 0  # ← 加入統一命名計數器

# 開始處理每張圖
for img_name in tqdm(os.listdir(input_dir)):
    img_path = os.path.join(input_dir, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)

    if img is None:
        continue  # 防呆

    h, w, _ = img.shape
    if h < PATCH_SIZE or w < PATCH_SIZE:
        continue  # 太小不處理

    # 對圖片做 patchify
    patches = patchify(img, (PATCH_SIZE, PATCH_SIZE, 3), step=STRIDE)

    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            patch = patches[i, j, 0]

            patch_name = f"{count:05d}.png"  # ← 改成純數字命名
            sharp_path = os.path.join(sharp_out_dir, patch_name)
            blur_path = os.path.join(blur_out_dir, patch_name)

            # 儲存清晰版本
            cv2.imwrite(sharp_path, patch)

            # 套用 Gaussian 模糊後儲存模糊版本
            blurred_patch = cv2.GaussianBlur(patch, GAUSSIAN_KERNEL, sigmaX=SIGMA_X)
            cv2.imwrite(blur_path, blurred_patch)

            count += 1  # ← 記得遞增
print("✅ DONE：T91 patch 裁切與模糊完成（純數字命名）！")

  0%|          | 0/91 [00:00<?, ?it/s]

100%|██████████| 91/91 [00:01<00:00, 60.11it/s] 

✅ DONE：T91 patch 裁切與模糊完成（純數字命名）！


#### (3)  Set5 與 Set14 資料集（驗證用）影像切割與模糊處理說明
**注意事項：**
- VAL_sharp_patches ： 存放清晰原始的影像 patch ， 作為驗證階段的標籤資料。
- VAL_blurred_patches : 存放經高斯模糊處理後的 patch ， 作為模型的輸入資料。
- 每個輸出影像檔案皆採用統一的五位數純數字命名（如 00001.png），這種命名方式有助於PyTorch 資料讀取流程 。

In [ ]:

# 設定來源與輸出資料夾
DIR_PATH = "../"  # 如果你的 notebook 和 inputs 是平行的，就用 ../，否則請依實際路徑修改
input_paths = [
    DIR_PATH + "inputs/Set5/",
    DIR_PATH + "inputs/Set14/"
]
val_sharp_dir = DIR_PATH + "inputs/VAL_sharp_patches"
val_blur_dir = DIR_PATH + "inputs/VAL_blurred_patches"

# 建立資料夾
os.makedirs(val_sharp_dir, exist_ok=True)
os.makedirs(val_blur_dir, exist_ok=True)

# 開始切 patch + 模糊
count = 0
for dir_path in input_paths:
    for filename in tqdm(os.listdir(dir_path), desc=f"處理 {os.path.basename(dir_path)}"):
        img_path = os.path.join(dir_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ 無法讀取：{img_path}")
            continue

        patches = patchify(img, (PATCH_SIZE, PATCH_SIZE, 3), step=STRIDE)
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                patch = patches[i, j, 0]
                patch_filename = f"{count:05d}.png"

                # 儲存清晰 patch
                sharp_path = os.path.join(val_sharp_dir, patch_filename)
                cv2.imwrite(sharp_path, patch)

                # 儲存模糊 patch
                blur_patch = cv2.GaussianBlur(patch, GAUSSIAN_KERNEL, sigmaX=SIGMA_X)
                blur_path = os.path.join(val_blur_dir, patch_filename)
                cv2.imwrite(blur_path, blur_patch)

                count += 1

print(f"✅ 成功產生 {count} 張驗證 patch 到：")
print(f"   {val_sharp_dir}")
print(f"   {val_blur_dir}")

處理 : 100%|██████████| 14/14 [00:01<00:00, 12.56it/s]

✅ 成功產生 352 張驗證 patch 到：
   ../inputs/VAL_sharp_patches
   ../inputs/VAL_blurred_patches


#### **<font color=cornflowerblue>學生與老師兩種版本比較表：</font>**   
| 處理面向                       | 學生版本              | 老師版本             |
|----------------------------|----------------------------------------|-------------------------------------------|
| **裁切大小 (Patch Size)**       | 96 × 96                                  | 33 × 33                                     |
| **裁切步長 (Stride)**           | 96                                       | 14                                          |
| **模糊核心大小 (Gaussian Kernel)** | (5, 5)                                    | (5, 5)                                      |
| **模糊程度 (Sigma X)**          | 0（自動推算）                            |0（自動推算）                              |
| **模糊順序**                   |  模糊套用於每個 patch（先裁切再模糊）     |  模糊也套用於每個 patch（先裁切再模糊） |
| **Patch 命名方式**             | 單純流水號（如 00001.png）               | 檔名＋編號（如 img_001.png）               |
| **資料完整性**       | 分開處理 T91 訓練集與 Set5/Set14 驗證集，並明確指定輸出資料夾    | 只處理 T30 資料，無驗證資料夾，未區分訓練／驗證用途             |
